# Laboratorio II - Análisis de Discriminante y Regresión Logística

In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import load_breast_cancer 
from sklearn.linear_model import LogisticRegression

## Base Pokémon

### General

In [48]:
pokemon = pd.read_csv('pokemon.csv')

In [49]:
pokemon.shape

(801, 41)

In [50]:
pokemon.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


Se utilizarán sólo los pokémon que tengan un solo tipo o el mismo tipo en ambos features.

In [51]:
pokemon_df = pokemon[(pokemon["type2"].isnull()) | (pokemon["type1"] == pokemon["type2"])]
pokemon_df.shape

(391, 41)

Se generarán dos modelos, uno que considera las 6 features planteadas en el pokémon de ejemplo de la segunda pregunta del laboratorio, y otro con las 34 features numéricas del DataFrame.

In [52]:
features = [
    "sp_attack",
    "sp_defense",
    "attack",
    "defense",
    "speed",
    "hp",
    "type1"
]
pokemon_6_features = pokemon_df[features]
pokemon_34_features = pokemon_df.drop(["abilities", "capture_rate", "classfication", "japanese_name", "name", "type2"], axis = 1)
print(pokemon_6_features.shape)
print(pokemon_34_features.shape)

(391, 7)
(391, 35)


### Roca y Hierba

- ¿Un pokémon tipo *roca* cómo se diferencia respecto de un pokémon tipo *hierba*?

Comparación de valores entre tipo *roca* y *hierba*.

In [53]:
feature_means1 = pd.DataFrame(columns=pokemon_6_features["type1"].drop_duplicates())
for c, rows in pokemon_6_features.groupby('type1'):
    feature_means1[c] = rows.mean()
# feature_means1
feature_means1[["rock", "grass"]]

/var/folders/vg/rrvy5r0s275_mqld96rv7s9r0000gn/T/ipykernel_10361/3099473545.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  feature_means1[c] = rows.mean()


type1,rock,grass
sp_attack,40.909091,71.815789
sp_defense,58.181818,63.842105
attack,100.909091,69.710526
defense,98.181818,68.894737
speed,39.818182,61.447368
hp,66.727273,63.184211


- Los promedios de los atributos especiales de ataque y defensa de los pokémon de tipo *roca* son menores a los de tipo *hierba*.
- Los promedios de atributos de ataque y defensa regular de los pokémon de tipo *roca* son cerca de un 44% mayores a los de tipo *hierba*.
- Los pokémon de tipo *roca* son en promedio un 35% más lentos que los de tipo *hierba*.
- Los pokémon de tipo roca tienen en promedio cerca de un 5% más de *hp*.

### 6 Features

In [54]:
pokemon_6_features["type1"].value_counts()

normal      61
water       61
grass       38
psychic     35
fire        28
electric    27
fighting    22
bug         18
fairy       16
poison      15
ground      12
dragon      12
ice         12
rock        11
dark         9
ghost        9
steel        4
flying       1
Name: type1, dtype: int64

Antes de estandaizar los valores y separar en conjuntos de entrenamiento y prueba, se eliminan las filas que contengan valores nulos.

In [55]:
pokemon_6_features.dropna(inplace = True)
X = pokemon_6_features.drop(["type1"], axis = 1)
y = pokemon_6_features["type1"]
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

/var/folders/vg/rrvy5r0s275_mqld96rv7s9r0000gn/T/ipykernel_10361/336929892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_6_features.dropna(inplace = True)


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lda_model = LinearDiscriminantAnalysis(n_components = 6)
X_lda = lda_model.fit_transform(X_train, y_train)

In [57]:
lda_model.explained_variance_ratio_

array([0.53580135, 0.17220053, 0.124615  , 0.09345872, 0.04913956,
       0.02478484])

In [58]:
pred_train = lda_model.predict(X_train)

In [59]:
print(confusion_matrix(y_train, pred_train))

[[ 2  0  0  1  0  0  0  0  0  0  1  0  3  0  0  0  0  4]
 [ 0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0  0  2]
 [ 0  0  0  0  2  4  0  0  0  0  0  0  0  0  0  0  0  4]
 [ 0  0  0  8  0  0  1  0  0  0  0  0  4  0  0  0  0  4]
 [ 0  0  0  0  4  0  0  0  0  0  0  0  2  0  3  0  0  5]
 [ 0  0  0  0  0 12  1  0  0  0  0  0  6  0  0  0  0  1]
 [ 0  0  1  0  0  1  3  0  0  0  0  0  4  0  1  0  0 11]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  2  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  2  1  1  2  0  0  0  1  0  2  0  1  0  0 15]
 [ 0  0  0  0  0  2  0  0  0  0  2  0  2  0  0  1  0  1]
 [ 0  0  0  0  1  0  1  0  1  0  1  0  0  0  0  0  0  3]
 [ 0  0  0  0  0  6  0  0  0  0  0  0 29  0  2  0  0 11]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  6  0  0  0  0  3]
 [ 0  0  0  2  1  0  1  0  1  0  0  0  3  0  7  0  0  3]
 [ 0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  3  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 1  0  0  1  0  0  0  0  1  0

In [60]:
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

         bug       0.67      0.18      0.29        11
        dark       0.00      0.00      0.00         5
      dragon       0.00      0.00      0.00        10
    electric       0.47      0.47      0.47        17
       fairy       0.44      0.29      0.35        14
    fighting       0.41      0.60      0.49        20
        fire       0.27      0.14      0.19        21
      flying       0.00      0.00      0.00         1
       ghost       0.40      0.40      0.40         5
       grass       0.00      0.00      0.00        25
      ground       0.40      0.25      0.31         8
         ice       0.00      0.00      0.00         7
      normal       0.42      0.60      0.50        48
      poison       0.00      0.00      0.00        10
     psychic       0.41      0.39      0.40        18
        rock       0.60      0.43      0.50         7
       steel       0.00      0.00      0.00         1
       water       0.31    

/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [61]:
pred_test = lda_model.predict(X_test)

In [62]:
print(confusion_matrix(y_test, pred_test))

[[ 2  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  3]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  1  0  1  0  0  1]
 [ 0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  0  2  0  0  5]
 [ 0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1]
 [ 0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  5]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  1]
 [ 0  0  0  1  0  0  0  0  0  0  1  0  1  0  0  0  0 10]
 [ 0  0  0  1  0  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  1  1  0  2]
 [ 0  0  0  0  0  3  0  0  1  0  0  0  6  0  1  0  0  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  4]
 [ 0  0  0  1  2  0  0  1  1  0  0  0  1  0  5  0  0  6]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  2  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  1  1  0  1  0  1  0

In [63]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

         bug       1.00      0.29      0.44         7
        dark       0.00      0.00      0.00         4
      dragon       0.00      0.00      0.00         2
    electric       0.43      0.30      0.35        10
       fairy       0.20      0.50      0.29         2
    fighting       0.00      0.00      0.00         2
        fire       0.33      0.14      0.20         7
      flying       0.00      0.00      0.00         0
       ghost       0.20      0.25      0.22         4
       grass       0.00      0.00      0.00        13
      ground       0.00      0.00      0.00         4
         ice       0.00      0.00      0.00         5
      normal       0.33      0.46      0.39        13
      poison       0.00      0.00      0.00         5
     psychic       0.36      0.29      0.32        17
        rock       0.50      0.50      0.50         4
       steel       0.00      0.00      0.00         3
       water       0.07    

/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [64]:
model_accuracy = round(accuracy_score(y_test, pred_test) * 100, 2)
print(f"Accuracy Model 6 Features: {model_accuracy}%")

Accuracy Model 6 Features: 20.34%


- Su modelo, ¿en qué tipo caracteriza un pokémon con sp_attack: 75.3, sp_defense: 51, attack: 63, defense: 47, speed: 97, hp: 66?

Ahora que el modelo está listo, se puede predecir el tipo de pokémon dadas las features utilizadas.

In [65]:
pokemon_test = {
    "sp_attack": 75.3,
    "sp_defense": 51,
    "attack": 63,
    "defense": 47,
    "speed": 97,
    "hp": 66
}

pokemon_test = pd.Series(pokemon_test)
pokemon_test = scaler.transform([pokemon_test])
lda_model.predict(pokemon_test)

/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array(['water'], dtype='<U8')

El pokémon entregado en la guía es clasificado como tipo *agua*.

### 34 Features

In [66]:
pokemon_34_features.head()

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,weight_kg,generation,is_legendary
3,0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,0.5,...,39,88.1,4,60,50,65,fire,8.5,1,0
4,0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,0.5,...,58,88.1,5,80,65,80,fire,19.0,1,0
6,1.0,1.0,1.0,2.0,1.0,1.0,0.5,1.0,1.0,2.0,...,44,88.1,7,50,64,43,water,9.0,1,0
7,1.0,1.0,1.0,2.0,1.0,1.0,0.5,1.0,1.0,2.0,...,59,88.1,8,65,80,58,water,22.5,1,0
8,1.0,1.0,1.0,2.0,1.0,1.0,0.5,1.0,1.0,2.0,...,79,88.1,9,135,115,78,water,85.5,1,0


In [67]:
pokemon_34_features.dropna(inplace = True)
X = pokemon_34_features.drop(["type1"], axis = 1)
y = pokemon_34_features["type1"]
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lda_model = LinearDiscriminantAnalysis(n_components = 15)
X_lda = lda_model.fit_transform(X_train, y_train)

In [69]:
lda_model.explained_variance_ratio_

array([9.22874809e-01, 4.62192419e-02, 1.63368113e-02, 8.02367651e-03,
       4.70190986e-03, 6.65182756e-04, 4.53739690e-04, 3.15429029e-04,
       2.14134442e-04, 9.86915831e-05, 4.34291343e-05, 3.44079753e-05,
       1.28917432e-05, 5.13746425e-06, 5.07658055e-07])

In [70]:
pred_train = lda_model.predict(X_train)

In [71]:
print(confusion_matrix(y_train, pred_train))

[[ 9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 21  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  8  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 37  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 19  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  7  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 43]]


In [72]:
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

         bug       1.00      1.00      1.00         9
        dark       1.00      1.00      1.00         5
      dragon       1.00      1.00      1.00         8
    electric       1.00      1.00      1.00        17
       fairy       1.00      1.00      1.00        11
    fighting       1.00      1.00      1.00        13
        fire       1.00      1.00      1.00        18
       ghost       1.00      1.00      1.00         8
       grass       1.00      1.00      1.00        21
      ground       1.00      1.00      1.00         7
         ice       1.00      1.00      1.00         8
      normal       1.00      1.00      1.00        37
      poison       1.00      1.00      1.00        11
     psychic       1.00      1.00      1.00        19
        rock       1.00      1.00      1.00         7
       water       1.00      1.00      1.00        43

    accuracy                           1.00       242
   macro avg       1.00   

In [73]:
pred_test = lda_model.predict(X_test)

In [74]:
print(confusion_matrix(y_test, pred_test))

[[ 9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 16  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 16  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 13]]


In [75]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

         bug       0.90      1.00      0.95         9
        dark       1.00      1.00      1.00         3
      dragon       1.00      1.00      1.00         4
    electric       1.00      1.00      1.00         5
       fairy       1.00      1.00      1.00         4
    fighting       1.00      1.00      1.00         9
        fire       1.00      1.00      1.00         9
      flying       0.00      0.00      0.00         1
       ghost       1.00      1.00      1.00         1
       grass       1.00      1.00      1.00        16
      ground       1.00      1.00      1.00         2
         ice       1.00      1.00      1.00         2
      normal       1.00      1.00      1.00        16
      poison       1.00      1.00      1.00         2
     psychic       1.00      1.00      1.00         6
        rock       1.00      1.00      1.00         2
       water       1.00      1.00      1.00        13

    accuracy              

/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthiascleinespinoza/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [76]:
model_accuracy = round(accuracy_score(y_test, pred_test) * 100, 2)
print(f"Accuracy Model 34 Features: {model_accuracy}%")

Accuracy Model 34 Features: 99.04%


## Base Breast Cancer

In [77]:
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target
cols = breast_cancer.feature_names

In [78]:
breast_cancer_df = pd.DataFrame(data = data, columns = cols)
breast_cancer_df["target"] = target

In [79]:
breast_cancer_df.shape

(569, 31)

In [80]:
breast_cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [81]:
bc_3_features = breast_cancer_df[["mean radius", 'mean concave points', "target"]]

In [82]:
X_breast_cancer = bc_3_features.drop(["target"], axis = 1)
y_breast_cancer= bc_3_features["target"]

In [84]:
X_train_breast_cancer, X_test_breast_cancer, y_train_breast_cancer, y_test_breast_cancer = train_test_split(X_breast_cancer, y_breast_cancer, test_size=0.2, random_state=4)

In [85]:
scaler = StandardScaler()
X_train_breast_cancer = scaler.fit_transform(X_train_breast_cancer.values)
X_test_breast_cancer = scaler.transform(X_test_breast_cancer.values)

In [86]:
logreg = LogisticRegression()
logreg.fit(X_train_breast_cancer, y_train_breast_cancer)

LogisticRegression()

In [87]:
y_pred_breast_cancer = logreg.predict(X_test_breast_cancer)

In [88]:
print(confusion_matrix(y_test_breast_cancer, y_pred_breast_cancer))

[[33  1]
 [ 8 72]]


In [89]:
print(classification_report(y_test_breast_cancer, y_pred_breast_cancer))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88        34
           1       0.99      0.90      0.94        80

    accuracy                           0.92       114
   macro avg       0.90      0.94      0.91       114
weighted avg       0.93      0.92      0.92       114



In [90]:
accuracy_score = round(accuracy_score(y_test_breast_cancer, y_pred_breast_cancer) * 100, 2)
print(f"Accuracy: {accuracy_score}%")

Accuracy: 92.11%


- Una mujer con un nódulo con radio medio de 15.5 ¿qué tan probable es que tenga cáncer?

Se debe considerar que 0 representa a *maligno* y 1 a *benigno*, por lo tanto las pacientes clasificadas como cero son las diagnosticadas con cancer.

In [91]:
bc_3_features[["mean radius", "target"]].groupby("target").describe()

mean radius                                                           
             count       mean       std     min     25%     50%    75%    max
target                                                                       
0            212.0  17.462830  3.203971  10.950  15.075  17.325  19.59  28.11
1            357.0  12.146524  1.780512   6.981  11.080  12.200  13.37  17.85

In [92]:
patient_probabilities = []

for i in np.arange(0, 30, 0.01):
    patient = {
        "mean radius": 15.5,
        "mean concave points": i
    }
    patient = pd.Series(patient)
    patient = scaler.transform([patient])
    patient_probabilities.append(logreg.predict_proba(patient)[:,0][0]*100)
sum(patient_probabilities)/len(patient_probabilities)

99.8477069839662

En promedio, la probabilidad de que una paciente con radio medio de 15.5 tenga cáncer es de un 99.85%. 

- ¿Cuántas veces más tiene una mujer con puntos cóncavos de 0.19 y radio medio de 16 que una mujer con puntos cóncavos de 0.25 y radio medio de 11.65 de tener cáncer?

In [21]:
patient_A = {
    "mean radius": 16, 
    'mean concave points': 0.19
}

patient_B = {
    "mean radius": 11.65, 
    'mean concave points': 0.25
}

patient_A = pd.Series(patient_A)
patient_B = pd.Series(patient_B)
patient_A = scaler.transform([patient_A])
patient_B = scaler.transform([patient_B])
prob_patient_A = logreg.predict_proba(patient_A)[:,0][0] * 100
prob_patient_B = logreg.predict_proba(patient_B)[:,0][0] * 100

print(prob_patient_A/prob_patient_B)

0.9999853913412792


Una mujer con puntos cóncavos de 0.19 y radio medio de 16 y otra con puntos cóncavos de 0.25 y radio medio de 11.65 tienen prácticamente la misma probabilidad de tener cáncer.